# Generación de Looks mediante Filtrado Progresivo de Triángulos

**Estrategia optimizada:**
1. Encontrar TODOS los triángulos válidos (respetando niveles de prendas)
2. Filtrar triángulos (NO nodos) progresivamente:
   - Filtro por PESO → Top 30% de triángulos
   - Filtro por CLUSTERING → Top 60% de esos triángulos
   - Filtro por PAGERANK → Top 50% de esos triángulos
3. Seleccionar los 10 mejores looks sin solapamiento de prendas

**Objetivo:** Generar 10+ looks válidos usando algoritmos de grafos vistos en teoría.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from time import time
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

## Configuración del Filtrado Progresivo

Ajusta estos porcentajes según necesites:

In [2]:
# ════════════════════════════════════════════════════════════
# CONFIGURACIÓN: Porcentajes de filtrado
# ════════════════════════════════════════════════════════════
FILTRO_PESO = 0.30        # Top 30% de triángulos por peso
FILTRO_CLUSTERING = 0.60  # Top 60% de esos por clustering
FILTRO_PAGERANK = 0.50    # Top 50% de esos por PageRank
N_LOOKS_FINALES = 10      # Número de looks que queremos

print("="*70)
print(" "*20 + "CONFIGURACIÓN")
print("="*70)
print(f"Filtro 1 (PESO):      Top {FILTRO_PESO*100:.0f}% de triángulos")
print(f"Filtro 2 (CLUSTERING): Top {FILTRO_CLUSTERING*100:.0f}% de triángulos")
print(f"Filtro 3 (PAGERANK):   Top {FILTRO_PAGERANK*100:.0f}% de triángulos")
print(f"Looks finales:        {N_LOOKS_FINALES}")
print("="*70)

                    CONFIGURACIÓN
Filtro 1 (PESO):      Top 30% de triángulos
Filtro 2 (CLUSTERING): Top 60% de triángulos
Filtro 3 (PAGERANK):   Top 50% de triángulos
Looks finales:        10


## 1. Carga del Grafo

In [3]:
print("="*70)
print("CARGANDO GRAFO")
print("="*70)

# IMPORTANTE: Ajusta la ruta según donde tengas tu archivo
t0 = time()
G = nx.read_gexf('grafo_prendas.gexf')
print(f"✓ Grafo cargado: {G.number_of_nodes()} prendas, {G.number_of_edges()} combinaciones")
print(f"  Tiempo: {time()-t0:.2f}s")
print()

# Extraer atributos de prendas
prendas_c = {}
for node in G.nodes():
    prendas_c[node] = G.nodes[node]

print(f"✓ Atributos de {len(prendas_c)} prendas cargados")
print("\nEjemplo de atributos (primer nodo):")
primer_nodo = list(G.nodes())[0]
for k, v in prendas_c[primer_nodo].items():
    print(f"  • {k}: {v}")

CARGANDO GRAFO
✓ Grafo cargado: 7555 prendas, 1140880 combinaciones
  Tiempo: 22.62s

✓ Atributos de 7555 prendas cargados

Ejemplo de atributos (primer nodo):
  • talla: 3
  • nivel: 2
  • sub_category: jersey
  • color_nivel: 54
  • long_cm: 58.0
  • sleeve_long_cm: 58.0
  • tiempo: warm
  • styles: classic, boho
  • prints: smooth
  • fits: loose
  • importance_score: 0.010333766500911052
  • versatilidad: 79.048
  • score_hits: 0.00035358236851880636
  • label: 1


## 2. Cálculo de Métricas Base

Calculamos las métricas que usaremos para filtrar triángulos.

**Algoritmos utilizados (vistos en teoría):**
- **Clustering coefficient** (Coeficiente de agrupamiento)
- **PageRank** (Centralidad por PageRank)
- **Triangles count** (Conteo de triángulos)

In [4]:
print("="*70)
print("CALCULANDO MÉTRICAS BASE")
print("="*70)
print()

# ════════════════════════════════════════════════════════════
# ALGORITMO 1: Clustering Coefficient
# ════════════════════════════════════════════════════════════
print(" Calculando Clustering Coefficient...")
t0 = time()
clustering = nx.clustering(G)
print(f"   ✓ Completado en {time()-t0:.2f}s")
print(f"   → Mide cohesión local de cada prenda")
print()

# ════════════════════════════════════════════════════════════
# ALGORITMO 2: PageRank
# ════════════════════════════════════════════════════════════
print(" Calculando PageRank...")
t0 = time()
pagerank = nx.pagerank(G, weight='weight', alpha=0.9)
print(f"   ✓ Completado en {time()-t0:.2f}s")
print(f"   → Mide importancia de cada prenda en el armario")
print()

# ════════════════════════════════════════════════════════════
# ALGORITMO 3: Triangle Count
# ════════════════════════════════════════════════════════════
print(" Calculando conteo de triángulos...")
t0 = time()
triangulos_dict = nx.triangles(G)
print(f"   ✓ Completado en {time()-t0:.2f}s")
print(f"   → Cuenta cuántos looks potenciales forma cada prenda")
print()

print("="*70)
print(" MÉTRICAS BASE CALCULADAS")
print("="*70)

CALCULANDO MÉTRICAS BASE

 Calculando Clustering Coefficient...
   ✓ Completado en 287.32s
   → Mide cohesión local de cada prenda

 Calculando PageRank...
   ✓ Completado en 2.74s
   → Mide importancia de cada prenda en el armario

 Calculando conteo de triángulos...
   ✓ Completado en 20.19s
   → Cuenta cuántos looks potenciales forma cada prenda

 MÉTRICAS BASE CALCULADAS


## 3. Búsqueda de Triángulos Válidos

Encontramos TODOS los triángulos que cumplen las reglas de niveles:
- Exactamente 1 prenda de nivel 1 (parte inferior)
- Las otras 2 prendas deben ser de niveles 2 o 3 (parte superior o complementos)

In [5]:
print("="*70)
print("BÚSQUEDA DE TRIÁNGULOS VÁLIDOS")
print("="*70)
print()

t0_total = time()

# Encontrar triángulos válidos
triangulos_validos = []
nodos_con_triangulos = [n for n, count in triangulos_dict.items() if count > 0]

print(f" Procesando {len(nodos_con_triangulos)} nodos con potencial de outfit...")
print()

# Iterar solo sobre nodos de nivel 1 (parte inferior)
nodos_nivel_1 = [n for n in nodos_con_triangulos if prendas_c[n]['nivel'] == 1]
print(f"   Nodos de nivel 1 (parte inferior): {len(nodos_nivel_1)}")

for n1 in nodos_nivel_1:
    vecinos = list(G.neighbors(n1))
    
    # Comprobar todas las parejas de vecinos
    for n2, n3 in itertools.combinations(vecinos, 2):
        # ¿Los vecinos están conectados entre sí? (triángulo)
        if G.has_edge(n2, n3):
            tri = [n1, n2, n3]
            
            # Verificar niveles
            niveles = [prendas_c[n]['nivel'] for n in tri]
            
            # Condiciones:
            # 1. Exactamente 1 prenda de nivel 1
            # 2. Todas las prendas de niveles válidos (1, 2, 3)
            # 3. Las otras 2 prendas deben ser nivel 2 o 3
            if niveles.count(1) == 1 and all(n in [1, 2, 3] for n in niveles):
                niveles_sin_1 = [n for n in niveles if n != 1]
                if all(n in [2, 3] for n in niveles_sin_1):
                    # Calcular peso (score promedio)
                    w12 = G[n1][n2]['weight']
                    w13 = G[n1][n3]['weight']
                    w23 = G[n2][n3]['weight']
                    score = round((w12 + w13 + w23) / 3, 4)
                    
                    triangulos_validos.append({
                        'ids': tri,
                        'peso_total': w12 + w13 + w23,
                        'score': score
                    })

print(f"\n TRIÁNGULOS ENCONTRADOS: {len(triangulos_validos):,}")
print(f"   Tiempo de búsqueda: {time()-t0_total:.2f}s")

if len(triangulos_validos) == 0:
    print("\n ERROR: No se encontraron triángulos válidos!")
    print("    Verifica que tu grafo tenga conexiones suficientes.")
else:
    pesos = [t['peso_total'] for t in triangulos_validos]
    print(f"   Peso máximo: {max(pesos):.4f}")
    print(f"   Peso mínimo: {min(pesos):.4f}")
    print(f"   Peso promedio: {np.mean(pesos):.4f}")

print("="*70)

BÚSQUEDA DE TRIÁNGULOS VÁLIDOS

 Procesando 7555 nodos con potencial de outfit...

   Nodos de nivel 1 (parte inferior): 2744

 TRIÁNGULOS ENCONTRADOS: 29,453,875
   Tiempo de búsqueda: 256.77s
   Peso máximo: 2.6400
   Peso mínimo: 0.5320
   Peso promedio: 1.4845


In [6]:
print("="*70)
print("VISUALIZACIÓN DE LOOKS GENERADOS")
print("="*70)
print()

print(f"📊 Procesando {len(triangulos_validos):,} triángulos válidos...")
print()

# Preparar datos para DataFrame
looks_data = []

for idx, triangulo in enumerate(triangulos_validos, 1):
    # Obtener IDs de las 3 prendas
    p1_id, p2_id, p3_id = triangulo['ids']
    
    # Obtener características de cada prenda
    p1 = prendas_c[p1_id]
    p2 = prendas_c[p2_id]
    p3 = prendas_c[p3_id]
    
    # Formatear estilos y estampados como strings
    p1_estilo = ', '.join(p1.get('styles', [])) if isinstance(p1.get('styles'), (list, set)) else str(p1.get('styles', ''))
    p2_estilo = ', '.join(p2.get('styles', [])) if isinstance(p2.get('styles'), (list, set)) else str(p2.get('styles', ''))
    p3_estilo = ', '.join(p3.get('styles', [])) if isinstance(p3.get('styles'), (list, set)) else str(p3.get('styles', ''))
    
    p1_print = ', '.join(p1.get('prints', [])) if isinstance(p1.get('prints'), (list, set)) else str(p1.get('prints', ''))
    p2_print = ', '.join(p2.get('prints', [])) if isinstance(p2.get('prints'), (list, set)) else str(p2.get('prints', ''))
    p3_print = ', '.join(p3.get('prints', [])) if isinstance(p3.get('prints'), (list, set)) else str(p3.get('prints', ''))
    
    # Crear fila con estructura exacta del documento
    looks_data.append({
        'prenda_1_id': p1_id,
        'prenda_1_tipo': p1.get('sub_category', 'N/A'),
        'prenda_1_color': p1.get('color_nivel', 'N/A'),
        'prenda_1_estampado': p1_print,
        'prenda_1_estilo': p1_estilo,
        'prenda_1_talla': p1.get('talla', 'N/A'),
        
        'prenda_2_id': p2_id,
        'prenda_2_tipo': p2.get('sub_category', 'N/A'),
        'prenda_2_color': p2.get('color_nivel', 'N/A'),
        'prenda_2_estampado': p2_print,
        'prenda_2_estilo': p2_estilo,
        'prenda_2_talla': p2.get('talla', 'N/A'),
        
        'prenda_3_id': p3_id,
        'prenda_3_tipo': p3.get('sub_category', 'N/A'),
        'prenda_3_color': p3.get('color_nivel', 'N/A'),
        'prenda_3_estampado': p3_print,
        'prenda_3_estilo': p3_estilo,
        'prenda_3_talla': p3.get('talla', 'N/A'),
    })
    
    # Mostrar progreso cada 1M de triángulos
    if idx % 1_000_000 == 0:
        print(f"   Procesados {idx:,} / {len(triangulos_validos):,} triángulos...")

# Crear DataFrame
df_looks = pd.DataFrame(looks_data)

print(f"\n✅ {len(triangulos_validos):,} triángulos procesados")
print(f"   Columnas: {len(df_looks.columns)}")
print()

# Configurar pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 30)
pd.set_option('display.width', 250)

# Mostrar las primeras filas
print("="*70)
print("PRIMERAS FILAS DEL DATAFRAME:")
print("="*70)
print(df_looks.head(10))
print()

# Estadísticas
print("="*70)
print("ESTADÍSTICAS:")
print("="*70)
print(f"Total de looks: {len(df_looks):,}")
print()
print("Columnas incluidas:")
for col in df_looks.columns:
    print(f"  • {col}")
print("="*70)

VISUALIZACIÓN DE LOOKS GENERADOS

📊 Procesando 29,453,875 triángulos válidos...

   Procesados 1,000,000 / 29,453,875 triángulos...
   Procesados 2,000,000 / 29,453,875 triángulos...
   Procesados 3,000,000 / 29,453,875 triángulos...
   Procesados 4,000,000 / 29,453,875 triángulos...
   Procesados 5,000,000 / 29,453,875 triángulos...
   Procesados 6,000,000 / 29,453,875 triángulos...
   Procesados 7,000,000 / 29,453,875 triángulos...
   Procesados 8,000,000 / 29,453,875 triángulos...
   Procesados 9,000,000 / 29,453,875 triángulos...
   Procesados 10,000,000 / 29,453,875 triángulos...
   Procesados 11,000,000 / 29,453,875 triángulos...
   Procesados 12,000,000 / 29,453,875 triángulos...
   Procesados 13,000,000 / 29,453,875 triángulos...
   Procesados 14,000,000 / 29,453,875 triángulos...
   Procesados 15,000,000 / 29,453,875 triángulos...
   Procesados 16,000,000 / 29,453,875 triángulos...
   Procesados 17,000,000 / 29,453,875 triángulos...
   Procesados 18,000,000 / 29,453,875 triáng

In [ ]:
df_looks.to_csv("Datos/Transformados/looks.csv")

## 4. Filtrado Progresivo de Triángulos

**Estrategia:**
1. **Filtro por PESO**: Seleccionar top 30% de triángulos con mayor peso
2. **Filtro por CLUSTERING**: De esos, seleccionar top 60% con mayor clustering promedio
3. **Filtro por PAGERANK**: De esos, seleccionar top 50% con mayor PageRank promedio

**IMPORTANTE:** Filtramos TRIÁNGULOS completos, no nodos individuales.

In [9]:
print("="*80)
print(" "*20 + "FILTRADO PROGRESIVO")
print("="*80)
print()

total_inicial = len(triangulos_validos)

# ════════════════════════════════════════════════════════════
# FILTRO 1: Por PESO (ya lo tenemos calculado)
# ════════════════════════════════════════════════════════════
print(f" FILTRO 1: Seleccionando top {FILTRO_PESO*100:.0f}% por PESO...")
t0 = time()

# Ordenar triángulos por peso total
sorted_by_peso = sorted(triangulos_validos, key=lambda x: x['peso_total'], reverse=True)
n_filtro1 = int(len(sorted_by_peso) * FILTRO_PESO)
triangulos_filtro1 = sorted_by_peso[:n_filtro1]

print(f"   De {total_inicial:,} → {len(triangulos_filtro1):,} triángulos ({len(triangulos_filtro1)/total_inicial*100:.1f}%)")
print()
print(f"   Tiempo: {time()-t0:.2f}s")

# ════════════════════════════════════════════════════════════
# FILTRO 2: Por CLUSTERING promedio
# ════════════════════════════════════════════════════════════
print(f" FILTRO 2: Seleccionando top {FILTRO_CLUSTERING*100:.0f}% por CLUSTERING...")
t0 = time()

# Calcular clustering promedio de cada triángulo
for tri in triangulos_filtro1:
    avg_clust = np.mean([clustering.get(nodo, 0) for nodo in tri['ids']])
    tri['clustering_avg'] = avg_clust

# Ordenar por clustering promedio
sorted_by_clustering = sorted(triangulos_filtro1, key=lambda x: x['clustering_avg'], reverse=True)
n_filtro2 = int(len(sorted_by_clustering) * FILTRO_CLUSTERING)
triangulos_filtro2 = sorted_by_clustering[:n_filtro2]

print(f"   De {len(triangulos_filtro1):,} → {len(triangulos_filtro2):,} triángulos ({len(triangulos_filtro2)/total_inicial*100:.1f}%)")
print(f"   Tiempo: {time()-t0:.2f}s")
print()

# ════════════════════════════════════════════════════════════
# FILTRO 3: Por PAGERANK promedio
# ════════════════════════════════════════════════════════════
print(f" FILTRO 3: Seleccionando top {FILTRO_PAGERANK*100:.0f}% por PAGERANK...")
t0 = time()

# Calcular PageRank promedio de cada triángulo
for tri in triangulos_filtro2:
    avg_pr = np.mean([pagerank.get(nodo, 0) for nodo in tri['ids']])
    tri['pagerank_avg'] = avg_pr

# Ordenar por PageRank promedio
sorted_by_pagerank = sorted(triangulos_filtro2, key=lambda x: x['pagerank_avg'], reverse=True)
n_filtro3 = int(len(sorted_by_pagerank) * FILTRO_PAGERANK)
triangulos_filtro3 = sorted_by_pagerank[:n_filtro3]

print(f"   De {len(triangulos_filtro2):,} → {len(triangulos_filtro3):,} triángulos ({len(triangulos_filtro3)/total_inicial*100:.1f}%)")
print(f"   Tiempo: {time()-t0:.2f}s")
print()

print("="*80)
print(f" RESUMEN DE FILTRADO:")
print(f"   Inicial:        {total_inicial:,} triángulos (100%)")
print(f"   Tras filtro 1:  {len(triangulos_filtro1):,} triángulos ({len(triangulos_filtro1)/total_inicial*100:.1f}%)")
print(f"   Tras filtro 2:  {len(triangulos_filtro2):,} triángulos ({len(triangulos_filtro2)/total_inicial*100:.1f}%)")
print(f"   Tras filtro 3:  {len(triangulos_filtro3):,} triángulos ({len(triangulos_filtro3)/total_inicial*100:.1f}%)")
print(f"   Reducción:      {100*(1-len(triangulos_filtro3)/total_inicial):.1f}%")
print("="*80)

                    FILTRADO PROGRESIVO

 FILTRO 1: Seleccionando top 30% por PESO...
   De 29,453,875 → 8,836,162 triángulos (30.0%)

   Tiempo: 95.43s
 FILTRO 2: Seleccionando top 60% por CLUSTERING...
   De 8,836,162 → 5,301,697 triángulos (18.0%)
   Tiempo: 81.36s

 FILTRO 3: Seleccionando top 50% por PAGERANK...
   De 5,301,697 → 2,650,848 triángulos (9.0%)
   Tiempo: 202.18s

 RESUMEN DE FILTRADO:
   Inicial:        29,453,875 triángulos (100%)
   Tras filtro 1:  8,836,162 triángulos (30.0%)
   Tras filtro 2:  5,301,697 triángulos (18.0%)
   Tras filtro 3:  2,650,848 triángulos (9.0%)
   Reducción:      91.0%


## 5. Selección Final de Looks

De los triángulos que sobrevivieron los 3 filtros, seleccionamos los 10 mejores **sin repetir prendas**.

In [10]:
def get_prefix_id(prenda_id):
    """
    Obtiene los dos primeros dígitos del ID de la prenda.
    Si el ID tiene menos de 4 dígitos, rellena con ceros a la izquierda.
    
    Ejemplos:
    - 7551 → '75'
    - 123 → '01' (asume que es 0123)
    - 45 → '00' (asume que es 0045)
    """
    # Convertir a string y rellenar con ceros a la izquierda hasta 4 dígitos
    id_str = str(prenda_id).zfill(4)
    
    # Retornar los dos primeros dígitos
    return id_str[:2]


print("="*80)
print(" "*25 + "SELECCIÓN FINAL")
print("="*80)
print()

# ════════════════════════════════════════════════════════════
# CALCULAR MAX_PESO UNA SOLA VEZ (fuera del loop)
# ════════════════════════════════════════════════════════════
print(" Calculando score final multi-criterio...")

#  CALCULAR UNA SOLA VEZ
max_peso = max([t['peso_total'] for t in triangulos_validos])
print(f"   Max peso calculado: {max_peso:.4f}")

# Ahora sí, iterar
for tri in triangulos_filtro3:
    # Normalizar peso
    peso_norm = tri['peso_total'] / max_peso  # ← Usa la variable ya calculada
    
    # Score compuesto (pesos ajustables)
    composite_score = (
        0.30 * peso_norm +                    # 30% creatividad (peso)
        0.30 * tri['clustering_avg'] +        # 30% cohesión
        0.40 * tri['pagerank_avg'] * 100      # 40% importancia
    )
    tri['score_final'] = composite_score

print(f"    Score calculado para {len(triangulos_filtro3):,} candidatos")
print()

# Ordenar por score final
print(" Ordenando candidatos...")
t0 = time()
candidatos_finales = sorted(triangulos_filtro3, key=lambda x: x['score_final'], reverse=True)
print(f"    Ordenados en {time()-t0:.2f}s")
print()

# ════════════════════════════════════════════════════════════
# SELECCIÓN SIN SOLAPAMIENTOS (MODIFICADO)
# ════════════════════════════════════════════════════════════
print(f" Seleccionando top {N_LOOKS_FINALES} looks SIN repetir prendas BASE...")
t0 = time()
looks_finales = []
prefijos_usados = set()  # ← Guardamos los prefijos (2 primeros dígitos) usados

for candidato in candidatos_finales:
    # Obtener los prefijos (2 primeros dígitos) de las 3 prendas del look
    prefijos_del_look = [get_prefix_id(prenda_id) for prenda_id in candidato['ids']]
    
    # ¿Algún prefijo ya fue usado?
    if not any(prefijo in prefijos_usados for prefijo in prefijos_del_look):
        looks_finales.append(candidato)
        prefijos_usados.update(prefijos_del_look)  # ← Marcar prefijos como usados
        
        if len(looks_finales) >= N_LOOKS_FINALES:
            break

print(f"    Seleccionados {len(looks_finales)} looks únicos")
print(f"    {len(prefijos_usados)} familias de prendas diferentes usadas")
print()
print("="*80)
print(f" GENERACIÓN COMPLETADA: {len(looks_finales)} LOOKS")
print("="*80)

                         SELECCIÓN FINAL

 Calculando score final multi-criterio...
   Max peso calculado: 2.6400
    Score calculado para 2,650,848 candidatos

 Ordenando candidatos...
    Ordenados en 5.20s

 Seleccionando top 10 looks SIN repetir prendas BASE...
    Seleccionados 10 looks únicos
    30 familias de prendas diferentes usadas

 GENERACIÓN COMPLETADA: 10 LOOKS


## 6. Visualización de Looks Generados

In [11]:
print("="*80)
print(" "*25 + "LOOKS GENERADOS")
print("="*80)
print()

for idx, look in enumerate(looks_finales, 1):
    print(f"{'═'*80}")
    print(f"LOOK #{idx:02d}")
    print(f"{'═'*80}")
    print(f"Prendas:           {look['ids']}")
    print(f"Peso total:        {look['peso_total']:.4f}")
    print(f"Score promedio:    {look['score']:.4f}")
    print(f"Clustering avg:    {look['clustering_avg']:.4f}")
    print(f"PageRank avg:      {look['pagerank_avg']:.6f}")
    print(f"Score final:       {look['score_final']:.4f}")
    print()
    
    # Interpretación
    if look['peso_total'] > 2.0:
        print(f" Look muy arriesgado/creativo")
    elif look['peso_total'] > 1.5:
        print(f" Look moderadamente arriesgado")
    else:
        print(f" Look conservador/seguro")
    
    if look['clustering_avg'] > 0.6:
        print(f" Prendas muy cohesivas entre sí")
    
    if look['pagerank_avg'] > np.median([l['pagerank_avg'] for l in looks_finales]):
        print(f" Usa prendas importantes del armario")
    
    print()

                         LOOKS GENERADOS

════════════════════════════════════════════════════════════════════════════════
LOOK #01
════════════════════════════════════════════════════════════════════════════════
Prendas:           ['7551', '2747', '3952']
Peso total:        2.5600
Score promedio:    0.8533
Clustering avg:    0.2878
PageRank avg:      0.000531
Score final:       0.3985

 Look muy arriesgado/creativo
 Usa prendas importantes del armario

════════════════════════════════════════════════════════════════════════════════
LOOK #02
════════════════════════════════════════════════════════════════════════════════
Prendas:           ['4633', '3548', '6812']
Peso total:        2.5600
Score promedio:    0.8533
Clustering avg:    0.2466
PageRank avg:      0.000456
Score final:       0.3831

 Look muy arriesgado/creativo

════════════════════════════════════════════════════════════════════════════════
LOOK #03
═════════════════════════════════════════════════════════════════════════

## 8. Exportación de Resultados

In [12]:
# ════════════════════════════════════════════════════════════
# Exportar looks a CSV con características de cada prenda
# ════════════════════════════════════════════════════════════
print("="*70)
print("EXPORTANDO LOOKS CON CARACTERÍSTICAS DE PRENDAS")
print("="*70)
print()

export_data = []

for idx, look in enumerate(looks_finales, 1):
    # Obtener IDs de las 3 prendas
    prenda_1_id = look['ids'][0]
    prenda_2_id = look['ids'][1]
    prenda_3_id = look['ids'][2]
    
    # Obtener características de cada prenda
    p1_attrs = prendas_c[prenda_1_id]
    p2_attrs = prendas_c[prenda_2_id]
    p3_attrs = prendas_c[prenda_3_id]
    
    # Crear fila con toda la información
    export_data.append({
        # Información del look
        'look_id': idx,
        
        # Métricas del look
        'peso_total': look['peso_total'],
        'score_promedio': look['score'],
        'clustering_avg': look['clustering_avg'],
        'pagerank_avg': look['pagerank_avg'],
        'score_final': look['score_final'],
        
        # PRENDA 1
        'prenda_1_id': prenda_1_id,
        'prenda_1_tipo': p1_attrs.get('sub_category', 'N/A'),
        'prenda_1_color': p1_attrs.get('color_nivel', 'N/A'),
        'prenda_1_estampado': p1_attrs.get('prints', 'N/A'),
        'prenda_1_estilo': p1_attrs.get('styles', 'N/A'),
        'prenda_1_talla': p1_attrs.get('talla', 'N/A'),
        
        # PRENDA 2
        'prenda_2_id': prenda_2_id,
        'prenda_2_tipo': p2_attrs.get('sub_category', 'N/A'),
        'prenda_2_color': p2_attrs.get('color_nivel', 'N/A'),
        'prenda_2_estampado': p2_attrs.get('prints', 'N/A'),
        'prenda_2_estilo': p2_attrs.get('styles', 'N/A'),
        'prenda_2_talla': p2_attrs.get('talla', 'N/A'),
        
        # PRENDA 3
        'prenda_3_id': prenda_3_id,
        'prenda_3_tipo': p3_attrs.get('sub_category', 'N/A'),
        'prenda_3_color': p3_attrs.get('color_nivel', 'N/A'),
        'prenda_3_estampado': p3_attrs.get('prints', 'N/A'),
        'prenda_3_estilo': p3_attrs.get('styles', 'N/A'),
        'prenda_3_talla': p3_attrs.get('talla', 'N/A'),
    })

# Crear DataFrame y exportar
df_export = pd.DataFrame(export_data)
df_export.to_csv('looks_generados.csv', index=False)

print(f" {len(looks_finales)} looks guardados en 'looks_generados_optimizado.csv'")
print(f"   Columnas exportadas: {len(df_export.columns)}")
print()
print("Columnas incluidas:")
for col in df_export.columns:
    print(f"  • {col}")
print()
print("="*70)
print("PRIMERAS FILAS DEL CSV:")
print("="*70)
print(df_export.head(3))

EXPORTANDO LOOKS CON CARACTERÍSTICAS DE PRENDAS



 10 looks guardados en 'looks_generados_optimizado.csv'
   Columnas exportadas: 24

Columnas incluidas:
  • look_id
  • peso_total
  • score_promedio
  • clustering_avg
  • pagerank_avg
  • score_final
  • prenda_1_id
  • prenda_1_tipo
  • prenda_1_color
  • prenda_1_estampado
  • prenda_1_estilo
  • prenda_1_talla
  • prenda_2_id
  • prenda_2_tipo
  • prenda_2_color
  • prenda_2_estampado
  • prenda_2_estilo
  • prenda_2_talla
  • prenda_3_id
  • prenda_3_tipo
  • prenda_3_color
  • prenda_3_estampado
  • prenda_3_estilo
  • prenda_3_talla

PRIMERAS FILAS DEL CSV:
   look_id  peso_total  score_promedio  clustering_avg  pagerank_avg  score_final prenda_1_id prenda_1_tipo  prenda_1_color prenda_1_estampado prenda_1_estilo prenda_1_talla prenda_2_id prenda_2_tipo  prenda_2_color         prenda_2_estampado  \
0        1        2.56          0.8533        0.287801      0.000531     0.398481        7551    pantalones             112             floral   classic, boho              2        2